## 네이버 영화 리뷰를 활용한 감정 분석
> 1. KoNLP 2. tf-idf를 활용한 임베딩
- 가능하면 해당 데이터를 기반으로 간단한 웹페이지도 만들어보자 !
- NLP 주요 라이브러리 설치
- pip install konlpy
> 직렬화, 시리얼라이저(Serializer) : 객체 => 이진파일
- pip install joblib

In [1]:
import numpy as np
import pandas as pd
import re # 파이썬 내장 정규식 처리

from konlpy.tag import Okt # 문형분석기

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

### 데이터 불러오기

In [2]:
train_df = pd.read_table("data/ratings_train.txt")
test_df = pd.read_table("data/ratings_test.txt")

In [3]:
train_df.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


### 전처리

In [4]:
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [5]:
train_df = train_df.fillna(" ")
test_df = test_df.fillna(" ")

In [6]:
train_df["document"]

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [7]:
# 숫자 삭제
train_df["document"] = train_df["document"].apply(lambda x: re.sub(r"\d+", " ", x))
# x는 document의 row 한 행을 의미
train_df["document"]

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [8]:
# 정규식
# re.sub(r"\d+", " ", "123 456 하하하") # 숫자만 없애는 정규식 사용

### 임베딩

In [9]:
okt = Okt()
def tw_tokenizer(text): # 트위터에서 만든 문자분리기
    tokenizer_ko = okt.morphs(text)
    return tokenizer_ko

In [10]:
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9) 
# max_df: 머신러닝에서는 몇퍼센트 강도를 유지할지, 딥러닝에서는 몇퍼센트 강도를 끊을지 의미
tfidf_vect.fit(train_df["document"]) # 숫자로 가득찬 행렬처럼 (리스트의 리스트 형태로) 나옴
tfidf_matrix_train = tfidf_vect.transform(train_df["document"])

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 학습

In [11]:
lr = LogisticRegression(C=3.5, random_state=42)
lr.fit(tfidf_matrix_train, train_df["label"])

LogisticRegression(C=3.5, random_state=42)

### 예측

In [14]:
tfidf_matrix_test = tfidf_vect.transform(test_df["document"])

In [16]:
preds = lr.predict(tfidf_matrix_test)
accuracy_score(test_df["label"],preds)

0.8612

In [20]:
p_text = tfidf_vect.transform(["이 영화 별로인듯!"])
lr.predict(p_text)

array([0], dtype=int64)

## 01 모델 만들기
- 1. 데이터 전처리
- 2. 머신러닝 학습
- 3. 결과 확인
- 4. 모델을 직렬화 : 객체 => 이진파일

## 02 서비스 설계
- 1. input (특성변수) - 꽃잎너비,꽃잎길이,꽃받침너비,꽃받침길이
- 2. 데이터 전처리 & 머신러닝 학습
- 3. output (확률)

## 03 구현
- 1. UI (특성변수) /// https
- 2. 스프링 서비스 /// rest api
- 3. 추론 서비스 (웹서비스)

## 04 구조
- model/(pkl파일), statics/, templates/, app.py

In [24]:
import joblib

In [27]:
joblib.dump(lr, "model/lr.pkl")
joblib.dump(tfidf_vect,"model/tfidf_vect.pkl")

['model/tfidf_vect.pkl']